In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
books = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Books.csv")

from pyspark.sql.functions import monotonically_increasing_id 

identifiedBooks = books.withColumn('bookId', monotonically_increasing_id().cast('int'));
identifiedBooks = identifiedBooks.select(['bookId', 'ISBN'])
identifiedBooks.printSchema()
identifiedBooks.show(10)

root
 |-- bookId: integer (nullable = false)
 |-- ISBN: string (nullable = true)

+------+----------+
|bookId|      ISBN|
+------+----------+
|     0|0195153448|
|     1|0002005018|
|     2|0060973129|
|     3|0374157065|
|     4|0393045218|
|     5|0399135782|
|     6|0425176428|
|     7|0671870432|
|     8|0679425608|
|     9|074322678X|
+------+----------+
only showing top 10 rows



In [3]:
users = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Users.csv")

users = users.withColumn('userId', users['User-ID'].cast('int'))
users.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [4]:
ratings = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Book-Ratings.csv")

ratings = ratings.withColumn('rating', ratings['Book-Rating'].cast('float'))

ratings = ratings.join(identifiedBooks,\
    ratings['ISBN'] == identifiedBooks['ISBN'],\
    'inner').drop(identifiedBooks['ISBN'])

ratings = ratings.join(users,\
    ratings['User-ID'] == users['User-ID'],\
    'inner').drop(users['User-ID'])
                             
ratings.printSchema()
                             
print("Pocet hodnoceni = %i" % ratings.count())

ratings.show(10)


root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = false)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)

Pocet hodnoceni = 1031175
+-------+----------+-----------+------+------+--------------------+----+------+
|User-ID|      ISBN|Book-Rating|rating|bookId|            Location| Age|userId|
+-------+----------+-----------+------+------+--------------------+----+------+
| 276725|034545104X|          0|   0.0|  2966|   tyler, texas, usa|NULL|276725|
| 276726|0155061224|          5|   5.0| 82022|seattle, washingt...|NULL|276726|
| 276727|0446520802|          0|   0.0| 11054|h, new south wale...|  16|276727|
| 276729|052165615X|          3|   3.0|103047|rijeka, n/a, croatia|  16|276729|
| 276729|0521795028|          6|   6.0|103048|rijeka, n/a, croatia|  16|276729|
| 276733|2080674722

In [7]:
ratings.write.format("parquet").mode("overwrite").save("/data/books/ratings-all.parquet")

In [10]:
train, test = ratings.randomSplit([0.7, 0.3])

train.write.format("parquet").mode("overwrite").save("/data/books/ratings-train.parquet")
test.write.format("parquet").mode("overwrite").save("/data/books/ratings-test.parquet")